#### Datathon 2022 - EPI Gijón

In [105]:
## Imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [106]:
## Parsing CSV data

df_amazon = pd.read_csv('data/Amazon.csv')
df_Email = pd.read_csv('data/Email.csv')
df_Facebook = pd.read_csv('data/Facebook.csv')
df_Google = pd.read_csv('data/Google.csv')
df_Instagram = pd.read_csv('data/Instagram.csv')
df_Radio = pd.read_csv('data/Radio.csv')
df_TV = pd.read_csv('data/TV.csv')
df_Youtube = pd.read_csv('data/Youtube.csv')


In [107]:
df_amazon.head(10)

,Date,Amazon
0,01-03-20,3408.66
1,01-04-20,36451.72
2,01-05-20,22675.68
3,01-06-20,25839.62
4,01-07-20,11967.21
5,01-08-20,21218.74
6,01-09-20,15664.69
7,01-10-20,13259.18
8,01-11-20,18210.74
9,01-12-20,24237.72


In [108]:
# Sumar quincenas de Faceboook

df_Facebook['Facebook'] = df_Facebook['Facebook 1 quincena'] + df_Facebook['Facebook 2 quincena']

df_Facebook.drop(columns='Facebook 1 quincena', inplace=True)
df_Facebook.drop(columns='Facebook 2 quincena', inplace=True)
df_Facebook.head(10)

,Date,Facebook
0,01-03-20,0.00
1,01-04-20,73571.23
2,01-05-20,0.00
3,01-06-20,0.00
4,01-07-20,0.00
5,01-08-20,0.00
6,01-09-20,0.00
7,01-10-20,0.00
8,01-11-20,0.00
9,01-12-20,0.00


In [112]:
df = pd.concat([df_Email])
df

,Date,Email
0,01-03-20,9816.14
1,01-04-20,15376.08
2,01-05-20,12172.60
3,01-06-20,10288.04
4,01-07-20,5420.80
5,01-08-20,7438.48
6,01-09-20,9093.15
7,01-10-20,6264.78
8,01-11-20,13470.34
9,01-12-20,8482.10
